In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Softmax
import matplotlib.pyplot as plt
import networkx as nx
from scipy.stats import ks_2samp
from graph_utils import avg_path_length
from graph_utils import get_cluster_coeff
from statsmodels import ztest as ztest

ModuleNotFoundError: No module named 'statsmodels'

In [2]:
#Run me

nA = 5
class Node:
    '''kekw'''
    def __init__(self, Q):
        self.Q = Q #Q-func
        self.E = [] #Experience pool

def initQ(in_size, i, lr):
    '''Initializes Q-approximator for 
     - in_size : Size of input (should be number of states + number of actions)
     - i : 
    '''
    input_layer = Input(shape = (in_size,))
    dense_1 = Dense(128, activation = 'relu', use_bias = True, kernel_initializer="glorot_uniform")(input_layer)
    dense_2 = Dense(128, activation = 'relu', use_bias = True, kernel_initializer="glorot_uniform")(dense_1)
    dense_3 = Dense(64, activation = 'relu', use_bias = True, kernel_initializer="glorot_uniform")(dense_2)
    o = Dense(1, activation = None, use_bias = True, kernel_initializer="glorot_uniform")(dense_3)
    
    Q = Model(input_layer, o, name = "Q" + str(i))

#     def mse(y_true, y_pred):
#         return tf.math.reduce_mean(tf.math.square(y_true - y_pred)) * 0.5
    
    optimizer = tf.keras.optimizers.SGD(learning_rate = lr, momentum = 0.0)
    loss =  tf.keras.losses.MeanSquaredError()

    Q.compile(loss = loss, optimizer = optimizer)
                           
    return Q

class World:
    def __init__(self, n, num_agents):
        #self.state = np.zeros((num_agents, n * n)) #State, 46 x 100
        self.state_nums = np.zeros((num_agents,), dtype = int)
        self.n = n
        self.G = nx.Graph()
        self.G.add_nodes_from(range(num_agents))
        self.num_agents = num_agents
        
        for m in range(num_agents): #Start agents in random states
            r = np.random.randint(0, n * n)
            #self.state[m, r] = 1
            self.state_nums[m] = r
    
    def get_pos(self):
        out = []
        for i in range(self.num_agents):
            loc = self.state_nums[i]
            y, x = loc // self.n, loc % self.n
            x += np.random.uniform()
            y += np.random.uniform()
            out.append((x, y))
        return dict(enumerate(out, start = 0))
        
    def update_G(self):
        '''
        Searches for agents in the same state, and adds one to their edge's weight
        Also decreases any existing connections that are not maintained's weights by 1
        Removes any connections that have weight leq to 0
        '''
        updated = []
        rewards = np.zeros((self.num_agents, 1))
        for ag in range(self.state_nums.shape[0]): 
            #TODO: Maybe replace with np where to add edges?
            for tg in range(self.state_nums.shape[0]):
                #If same state, create/add weight to edge
                if ag != tg and self.state_nums[ag] == self.state_nums[tg] and (tg, ag) not in updated: 
                    if not self.G.has_edge(ag, tg):            #^ that fixes the double counhting but if i leave it
                        self.G.add_edge(ag, tg, weight = 1)  #it has better time complexity
                    else:
                        self.G.edges[ag, tg]['weight'] += 1 #<- thats why theyre 0.5 
                    updated.append((ag, tg))
                    updated.append((tg, ag))
                    
        for edge in self.G.edges():
            #Decrement all unupdated edges by 1, remove if less than 0
            if edge not in updated:
                self.G.edges[edge]['weight'] -= 0.025
                if self.G.edges[edge]['weight'] <= 0:
                    self.G.remove_edge(edge[0], edge[1])
                else:
                    rewards[edge[0]] += self.G.edges[edge]['weight']
                    rewards[edge[1]] += self.G.edges[edge]['weight']
            else:
                rewards[edge[0]] += self.G.edges[edge]['weight']
                rewards[edge[1]] += self.G.edges[edge]['weight']
            
        return rewards
    
                    
    def take_action(self, agent_id, a):
        #loc = np.argmax(self.state[agent_id, :])
        loc = self.state_nums[agent_id]
        y, x = loc // self.n, loc % self.n
        
        if a == 4: #down
            #self.state[agent_id, loc] = 0
            #self.state[agent_id, ((y + 1) * self.n) + x] = 1
            self.state_nums[agent_id] = ((y + 1) * self.n) + x
        elif a == 1: #up
           # self.state[agent_id, loc] = 0
            #self.state[agent_id, ((y - 1) * self.n) + x] = 1
            self.state_nums[agent_id] = ((y - 1) * self.n) + x
        elif a == 2: #left
            #self.state[agent_id, loc] = 0
            #self.state[agent_id, ((y) * self.n) + x - 1] = 1
            self.state_nums[agent_id] = ((y) * self.n) + x - 1
        elif a == 3: #right
            #self.state[agent_id, loc] = 0
            #self.state[agent_id, ((y) * self.n) + x + 1] = 1  
            self.state_nums[agent_id] = ((y) * self.n) + x + 1
        elif a == 0: #stay
            pass

def get_legals(s, n, nA):
    #Returns an array containig infinity if legal negative infinity if not
    legal_moves = np.ones((nA, 1)) * np.inf
    loc = s #np.argmax(s)
    y, x = loc // n, loc % n
    if y >= n - 1:
        legal_moves[4] = -np.inf
    if y == 0:
        legal_moves[1] = -np.inf
    if x == 0:
        legal_moves[2] = -np.inf
    if x == n - 1:
        legal_moves[3] = -np.inf

    return legal_moves
    
    #Make the graph pretty
def pp_graph(GRAPH_NAME, pos):
    #not short for pee pee graph
    widths = nx.get_edge_attributes(GRAPH_NAME, 'weight')
    nodelist = GRAPH_NAME.nodes()

    plt.figure(figsize=(10,10))

    #pos = nx.spring_layout(GRAPH_NAME)
    nx.draw_networkx_nodes(GRAPH_NAME, pos,
                           nodelist=nodelist,
                           node_size=100,
                           alpha=0.7)
    nx.draw_networkx_edges(GRAPH_NAME, pos,
                           edgelist = widths.keys(),
                           width=[val*1 for val in list(widths.values())],
                           edge_color='black',
                           alpha=0.75)
    plt.box(False)
    plt.show() 

    
def form_2d_graph(snums):
    xout, yout = [], []
    for s in snums:
        y, x = s // n, s % n
        xout.append((x + np.random.uniform())* 10)
        yout.append((y + np.random.uniform()) * 10)
    
    return xout, yout

def plot_2d_graph(x, y):
    fig = plt.figure()
    ax = fig.gca()
    ax.set_xticks(np.arange(0, 100, 10))
    ax.set_yticks(np.arange(0, 100, 10))
    plt.scatter(x, y, s=10)
    plt.grid()
    plt.show()
    
def degree_distribution(graph, name = "bruh"):
    degree_sequence = sorted((d for _, d in graph.degree()), reverse=True)

    plt.figure(name, figsize=(8, 8))
    plt.bar(*np.unique(degree_sequence, return_counts=True))
    plt.xlabel('Degree')
    plt.ylabel('# of Nodes')

    plt.show()

    

In [3]:
def monk_it_up(epsilon_func, episode_length = 100, n = 10, y = 0.9, nAgents = 47, lr = 0.001, verbose = 1):
    '''
    Inputs:
     - epsilon_func: Function taht takes a timestep first, and then an agent id i, and returns a
                     epsilon between 0 and 1, keep in mind t starts at 0
     - episode_length: Number of steps in an episode
     - n: dimensions of world, will be nxn size
     - y: discount factor
     - nAgents: number of monkeys
     - lr: Learning rate, controls how fast neural net learns. Careful with putting lr too high, since 0.01 makes 
           gradient bounce around super fast, leading to a lot of nan predictions
     - verbose: 
         - if 0: prints nothing
         - if 1: prints Z-score and KSamps
         - if 2: prints graphs, z-score and Ksamps
           
    Returns:
    world object: Object representing the world, do world.G to get the graph, or world.state_nums for current board
                  state
    Avg path length list: List containing average path legnths every 10 timesteps (CURRENTLY NANS)
    clsuter_avgs: List containing average clustering coefficient every timestep
    zs : Numpy array of z-scores at each timestep (compared to our proximity graph, lower is better)
    samps : listo f KS-2  samples(tuples) at every timestep
    hist_gs : Graph of every timestep
    '''
    p_lengths = []
    cluster_avgs = []
    zs = []
    samps = []
    hist_gs = []
    
    worse_graph = nx.read_adjlist("./prog_g.adjlist")
    w_degree_sequence = np.unique(sorted((d for _, d in worse_graph.degree()), reverse=True), return_counts = True)[1]
    
    all_actions = np.eye(nA, dtype = np.float32)
    state = np.zeros((nA, n * n))
    vec = np.concatenate((state, all_actions), axis = 1)

    def policy(s, n, nA, epsilon, Q):
        legal_moves = get_legals(s, n, nA)
        #Epsilon exploration
        if np.random.uniform() < epsilon:
            r = np.random.randint(0, nA)
            while legal_moves[r] != np.inf:
                r = np.random.randint(0, nA)
            return r

        vec[:, s] = 1    
        out = np.argmax(np.minimum(Q(vec), legal_moves))
        vec[:, s] = 0

        return out

    upd_vec = np.zeros((1, n * n + nA))
    tg_vec = np.zeros((1, n * n + nA))

    def update(s, a, r, sp, gamma, Q):
        upd_vec[0, s] = 1
        upd_vec[0, s + a] = 1


        tg_vec[0, sp] = 1
        ap = policy(sp, n, nA, 0, Q)
        tg_vec[0, ap + sp] = 1
        Q.train_on_batch(upd_vec, r + gamma * Q(tg_vec))
        upd_vec[0, s], upd_vec[0, s + a], tg_vec[0, sp], tg_vec[0, ap + sp]  = 0, 0, 0, 0
        
    agents = [Node(initQ(n * n + nA, i, lr)) for i in range(nAgents)]

    world = World(n, nAgents)

    for t in range(episode_length):
        if t % 10 == 0 and verbose >= 1:
            print(f"Iter : {t}")
        

        for agent, i in zip(agents, range(nAgents)):
            epsilon = epsilon_func(t, i)
            sars = np.zeros((4,), dtype = int)
            s = world.state_nums[i]
            a = policy(s, n, nA, epsilon, agent.Q)

            world.take_action(i, a)

            sars[0] = s
            sars[1] = a
            sars[3] = world.state_nums[i]

            agent.E.append(sars) #r will come later

        R = world.update_G()

        for agent, i in zip(agents, range(nAgents)):
            agent.E[t][2] = R[i]
            s, a, r, sp = agent.E[np.random.choice(len(agent.E))]
            #s, a, r, sp = agent.E[t]
            #update(s, a, r, sp, y, agent.Q)
            
        p_len = None
        if t % 5 == 0:
            p_len = avg_path_length(world.G)
            p_lengths.append(p_len)
        if t % 5 == 0 and verbose >= 2:
            pp_graph(world.G, world.get_pos())
            print(f"Average path length: {p_len}")
            
        
        degree_sequence = np.unique(sorted((d for _, d in world.G.degree()), reverse=True),
                                    return_counts = True)[1]
        top = (degree_sequence.mean() - w_degree_sequence.mean()) 
        bot1 = np.square(np.std(degree_sequence) / np.sqrt(degree_sequence.shape[0]))
        bot2 = np.square(np.std(w_degree_sequence)) / np.sqrt(w_degree_sequence.shape[0])
        z = top / np.sqrt(bot1 + bot2)
        

        zs.append(z)

        s = ks_2samp(degree_sequence, w_degree_sequence)
        samps.append(s)
        if t % 10 == 0 and verbose >= 1:
            print(f"Z-score : {z}")
            print(f"ks_2 sample : {s}")

        hist_gs.append(world.G.copy())
        cluster_avgs.append(nx.average_clustering(world.G))
        
    return world, p_lengths, cluster_avgs, np.array(zs), samps, hist_gs

In [4]:
def mem_friendly(epsilon_func, episode_length = 500, n = 7, y = 0.9, nAgents = 47, lr = 0.001, verbose = 1):
    '''
    Inputs:
     - epsilon_func: Function taht takes a timestep first, and then an agent id i, and returns a
                     epsilon between 0 and 1, keep in mind t starts at 0
     - episode_length: Number of steps in an episode
     - n: dimensions of world, will be nxn size
     - y: discount factor
     - nAgents: number of monkeys
     - lr: Learning rate, controls how fast neural net learns. Careful with putting lr too high, since 0.01 makes 
           gradient bounce around super fast, leading to a lot of nan predictions
     - verbose: 
         - if 0: prints nothing
         - if 1: prints Z-score and KSamps
         - if 2: prints graphs, z-score and Ksamps
           
    Returns:
    world object: Object representing the world, do world.G to get the graph, or world.state_nums for current board
                  state
    Avg path length list: List containing average path legnths every 10 timesteps (CURRENTLY NANS)
    clsuter_avgs: List containing average clustering coefficient every timestep
    zs : Numpy array of z-scores at each timestep (compared to our proximity graph, lower is better)
    samps : listo f KS-2  samples(tuples) at every timestep
    hist_gs : Graph of every timestep
    '''
    p_lengths = []
    cluster_avgs = []
    zs = []
    samps = []
    
    worse_graph = nx.read_adjlist("./prog_g.adjlist")
    w_degree_sequence = np.unique(sorted((d for _, d in worse_graph.degree()), reverse=True), return_counts = True)[1]
    
    all_actions = np.eye(nA, dtype = np.float32)
    state = np.zeros((nA, n * n))
    vec = np.concatenate((state, all_actions), axis = 1)

    def policy(s, n, nA, epsilon, Q):
        legal_moves = get_legals(s, n, nA)
        #Epsilon exploration
        if np.random.uniform() < epsilon:
            r = np.random.randint(0, nA)
            while legal_moves[r] != np.inf:
                r = np.random.randint(0, nA)
            return r

        vec[:, s] = 1    
        out = np.argmax(np.minimum(Q(vec), legal_moves))
        vec[:, s] = 0

        return out

    upd_vec = np.zeros((1, n * n + nA))
    tg_vec = np.zeros((1, n * n + nA))

    def update(s, a, r, sp, gamma, Q):
        upd_vec[0, s] = 1
        upd_vec[0, s + a] = 1


        tg_vec[0, sp] = 1
        ap = policy(sp, n, nA, 0, Q)
        tg_vec[0, ap + sp] = 1
        Q.train_on_batch(upd_vec, r + gamma * Q(tg_vec))
        upd_vec[0, s], upd_vec[0, s + a], tg_vec[0, sp], tg_vec[0, ap + sp]  = 0, 0, 0, 0
        
    agents = [Node(initQ(n * n + nA, i, lr)) for i in range(nAgents)]

    world = World(n, nAgents)

    for t in range(episode_length):
        if t % 10 == 0 and verbose >= 1:
            print(f"Iter : {t}")
        

        for agent, i in zip(agents, range(nAgents)):
            epsilon = epsilon_func(t, i)
            sars = np.zeros((4,), dtype = int)
            s = world.state_nums[i]
            a = policy(s, n, nA, epsilon, agent.Q)

            world.take_action(i, a)

            sars[0] = s
            sars[1] = a
            sars[3] = world.state_nums[i]

            agent.E.append(sars) #r will come later

        R = world.update_G()

        for agent, i in zip(agents, range(nAgents)):
            agent.E[t][2] = R[i]
            s, a, r, sp = agent.E[np.random.choice(len(agent.E))]
            #s, a, r, sp = agent.E[t]
            #update(s, a, r, sp, y, agent.Q)
            

        p_len = avg_path_length(world.G)
        p_lengths.append(p_len)
        if t % 5 == 0 and verbose >= 2:
            pp_graph(world.G, world.get_pos())
            print(f"Average path length: {p_len}")
            
        
        degree_sequence = np.unique(sorted((d for _, d in world.G.degree()), reverse=True),
                                    return_counts = True)[1]
        top = (degree_sequence.mean() - w_degree_sequence.mean()) 
        bot1 = np.square(np.std(degree_sequence) / np.sqrt(degree_sequence.shape[0]))
        bot2 = np.square(np.std(w_degree_sequence)) / np.sqrt(w_degree_sequence.shape[0])
        z = top / np.sqrt(bot1 + bot2)
        

        zs.append(z)

        s = ks_2samp(degree_sequence, w_degree_sequence)
        samps.append(s)
        if t % 10 == 0 and verbose >= 1:
            print(f"Z-score : {z}")
            print(f"ks_2 sample : {s}")

        cluster_avgs.append(nx.average_clustering(world.G))
        
    return p_lengths, cluster_avgs, np.array(zs), samps

In [5]:
nAgents = 47
eps_funcs = np.random.uniform(0.2, 0.5, (nAgents,))
e_fixed = lambda t, i: 0.3
e_fixed_2 = lambda t, i: 0.4
e_decay = lambda t, i : 1 / (t + 1)
e_brainoff = lambda t, i: 1

res = [0] * 3

#250 Iters, -0.025
for e, j in zip((e_fixed, e_brainoff), range(2)):
    res[j] = []
    for i in range(100):
        print(f"iter: {i}")
        pl, cl, z, samp = mem_friendly(e, 500, n  = 7,nAgents = 47, verbose = 0)
        res[j].append((pl, cl, z, samp))
        #pp_graph(res[i][0].G, res[i][0].get_pos())

iter: 0


2022-11-24 19:16:43.380070: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


iter: 1
iter: 2
iter: 3
iter: 4
iter: 5
iter: 6
iter: 7
iter: 8
iter: 9
iter: 10
iter: 11
iter: 12
iter: 13
iter: 14
iter: 15
iter: 16
iter: 17
iter: 18
iter: 19
iter: 20
iter: 21
iter: 22
iter: 23
iter: 24
iter: 25
iter: 26
iter: 27
iter: 28
iter: 29
iter: 30
iter: 31
iter: 32
iter: 33
iter: 34
iter: 35
iter: 36
iter: 37
iter: 38
iter: 39
iter: 40
iter: 41
iter: 42
iter: 43
iter: 44
iter: 45
iter: 46
iter: 47
iter: 48
iter: 49
iter: 50
iter: 51
iter: 52
iter: 53
iter: 54
iter: 55
iter: 56
iter: 57
iter: 58
iter: 59
iter: 60
iter: 61
iter: 62
iter: 63
iter: 64
iter: 65
iter: 66
iter: 67
iter: 68
iter: 69
iter: 70
iter: 71
iter: 72
iter: 73
iter: 74
iter: 75
iter: 76
iter: 77
iter: 78
iter: 79
iter: 80
iter: 81
iter: 82
iter: 83
iter: 84
iter: 85
iter: 86
iter: 87
iter: 88
iter: 89
iter: 90
iter: 91
iter: 92
iter: 93
iter: 94
iter: 95
iter: 96
iter: 97
iter: 98
iter: 99
iter: 0
iter: 1
iter: 2
iter: 3
iter: 4
iter: 5
iter: 6
iter: 7
iter: 8
iter: 9
iter: 10
iter: 11
iter: 12
iter: 13
it

In [6]:
res.pop(-1)

0

In [8]:
for iters in range(150, 500, 10):
    print(f"Iters: {iters}")
    for res_e in res:
        mins = []
        stat = []
        p = []
        cluss = []
        pplens = []
        for run in res_e:
            plens, clus_avgs, zs, samp = run
            mins.append(zs[iters])
            stat.append(samp[iters][0])
            p.append(samp[iters][1])
            cluss.append(clus_avgs[iters])
            pplens.append(plens[iters])

        print(f"Average most recent Z-score: {np.mean(mins)}")
        print(f"Average most recent stat-score: {np.mean(stat)}")
        print(f"Average most recent p-value: {np.mean(p)}")
        print(f"Average most recent clustering coeff: {np.mean(cluss)}")
        print(f"Average most recent path length: {np.mean(pplens)}")

Iters: 150
Average most recent Z-score: 1.0308504921462422
Average most recent stat-score: 0.30945414547537986
Average most recent p-value: 0.34792842148083997
Average most recent clustering coeff: 0.6047973530871337
Average most recent path length: 4.396623496762257
Average most recent Z-score: 1.2430014801531086
Average most recent stat-score: 0.34025053760309176
Average most recent p-value: 0.28178144518697407
Average most recent clustering coeff: 0.635695711115002
Average most recent path length: 4.21695189639223
Iters: 160
Average most recent Z-score: 0.97022008782117
Average most recent stat-score: 0.28572386488640356
Average most recent p-value: 0.42022221072043336
Average most recent clustering coeff: 0.6070806455415688
Average most recent path length: 4.5093802035152635
Average most recent Z-score: 1.2696008894355755
Average most recent stat-score: 0.3565475186578127
Average most recent p-value: 0.2352530815176551
Average most recent clustering coeff: 0.6455188825512768
Averag

In [ ]:
for res_e in res:    
    mins = []
    stat = []
    p = []
    cluss = []
    pplens = []
    for run in res_e:
        plens, clus_avgs, zs, samp = run
        mins.append(zs[-1])
        stat.append(samp[-1][0])
        p.append(samp[-1][1])
        cluss.append(clus_avgs[-1])
        pplens.append(plens[-1])

    print(f"Average most recent Z-score: {np.mean(mins)}")
    print(f"Average most recent stat-score: {np.mean(stat)}")
    print(f"Average most recent p-value: {np.mean(p)}")
    print(f"Average most recent clustering coeff: {np.mean(cluss)}")
    print(f"Average most recent path length: {np.mean(pplens)}")


    mins_b = []
    stat_b = []
    p_b = []
    cluss_b = []
    pplens_b = []    
    for run in res_e:
        plens, clus_avgs, zs, samp = run
        mins_b.append(np.min(zs))
        amin = np.argmax(zs) #Choose graph with most similar z-score
        stat_b.append(samp[amin][0])
        p_b.append(samp[amin][1])
        cluss_b.append(clus_avgs[amin])
        pplens_b.append(plens[amin // 5])

    print(f"Average best Z-score: {np.mean(mins_b)}")
    print(f"Average best stat-score: {np.mean(stat_b)}")
    print(f"Average best p-value: {np.mean(p_b)}")
    print(f"Average best clustering coeff: {np.mean(cluss_b)}")
    print(f"Average best path length: {np.mean(pplens_b)}")
    print("\n")

In [ ]:
nAgents = 47
eps_funcs = np.random.uniform(0.2, 0.5, (nAgents,))
e_dist = lambda t, i : dist[i]
e_fixed = lambda t, i: 0.3
e_brainoff = lambda t, i: 1
world, plens, clus_avgs, zs, samp, hist_gs = monk_it_up(e_brainoff, 300, n  = 10,nAgents = 47, verbose = 1)

In [ ]:
mins = []
stat = []
p = []
cluss = []
pplens = []
for run in res:
    world, plens, clus_avgs, zs, samp, hist_gs = run
    mins.append(zs[-1])
    stat.append(samp[-1][0])
    p.append(samp[-1][1])
    cluss.append(clus_avgs[-1])
    pplens.append(plens[-1])

In [ ]:
worse_graph = nx.read_adjlist("./prog_g.adjlist")
print(f"Generated: {avg_path_length(world.G)}")
avg_path_length(worse_graph)

In [ ]:
clus_avgs

In [ ]:
print(np.min(zs))
samp[np.argmin(zs)]

In [ ]:
plt.plot(zs)
plt.show()

In [ ]:
degree_distribution(hist_gs[a])

In [ ]:
degree_distribution(world.G)

In [ ]:
worse_graph = nx.read_adjlist("./prog_g.adjlist")
w_degree_sequence = np.unique(sorted((d for _, d in worse_graph.degree()), reverse=True), return_counts = True)[1]


degree_sequence = np.unique(sorted((d for _, d in world.G.degree()), reverse=True),
                                        return_counts = True)[1]
top = (degree_sequence.mean() - w_degree_sequence.mean()) 
bot1 = np.square(np.std(degree_sequence) / np.sqrt(degree_sequence.shape[0]))
bot2 = np.square(np.std(w_degree_sequence)) / np.sqrt(w_degree_sequence.shape[0])
print(f"Z-score : {top / np.sqrt(bot1 + bot2)}")

ks_2samp(degree_sequence, w_degree_sequence)
print(f"ks_2 sample : {ks_2samp(degree_sequence, w_degree_sequence)}")

In [ ]:
ks_2samp(degree_sequence, w_degree_sequence)

In [ ]:
plens